# t-SNE Visualization Analysis

This notebook displays results from the automated t-SNE analysis pipeline. All computations are handled by the pipeline - this notebook focuses on narrative and visualization.

In [ ]:
from core.config import initialize_notebook
import pickle
import matplotlib.pyplot as plt
from pathlib import Path

# Initialize environment
env = initialize_notebook(regenerate_run_id=False)
run_cfg = env.configs.run
data_dir = env.repo_root / "outputs" / run_cfg["run_name"] / run_cfg["run_id"] / f"seed_{run_cfg['seed']}"
embeddings_dir = data_dir / "tsne_embeddings"
plots_dir = embeddings_dir / "plots"

print(f"Loading t-SNE results from: {embeddings_dir}")

In [ ]:
# Run t-SNE analysis if not already computed
if not embeddings_dir.exists() or not (plots_dir / f"qc_comparison_complexity{env.configs.tsne['complexity']}.png").exists():
    print("t-SNE results not found. Running t-SNE analysis pipeline...")
    from core.tsne.pipeline import run_tsne_analysis
    results = run_tsne_analysis(env)
    print("t-SNE analysis complete!")
else:
    print("t-SNE results found, loading existing analysis")

## Quality Control Impact

The first analysis shows how quality control filtering affects the overall data distribution in t-SNE space. We expect to see outliers with high surface topology defects removed after QC.

In [ ]:
# Display QC comparison plot
complexity = env.configs.tsne["complexity"]
qc_plot = plots_dir / f"qc_comparison_complexity{complexity}.png"

if qc_plot.exists():
    from IPython.display import Image, display
    display(Image(str(qc_plot)))
else:
    print("QC comparison plot not found. Run the full pipeline first.")

## Harmonization Effectiveness

These plots demonstrate how harmonization reduces site effects while preserving biological signal of interest (anxiety groups).

In [ ]:
# Display harmonization impact on anxiety groups
anxiety_plot = plots_dir / f"harmonization_anxiety_complexity{complexity}.png"

if anxiety_plot.exists():
    display(Image(str(anxiety_plot)))
else:
    print("Anxiety harmonization plot not found. Run the full pipeline first.")

In [ ]:
# Display harmonization impact on scanner effects
scanner_plot = plots_dir / f"harmonization_scanner_complexity{complexity}.png"

if scanner_plot.exists():
    display(Image(str(scanner_plot)))
else:
    print("Scanner harmonization plot not found. Run the full pipeline first.")

## Demographics Distribution

These plots show how demographic variables (age, sex) are distributed in the t-SNE space before and after harmonization.

In [ ]:
# Display age distribution
age_plot = plots_dir / f"demographics_age_complexity{complexity}.png"

if age_plot.exists():
    display(Image(str(age_plot)))
else:
    print("Age demographics plot not found. Run the full pipeline first.")

In [ ]:
# Display sex distribution
sex_plot = plots_dir / f"demographics_sex_complexity{complexity}.png"

if sex_plot.exists():
    display(Image(str(sex_plot)))
else:
    print("Sex demographics plot not found. Run the full pipeline first.")

## Summary Statistics

Load and display key statistics from the analysis.

In [ ]:
# Load metadata to show summary stats
metadata_path = embeddings_dir / "metadata.pkl"

if metadata_path.exists():
    with open(metadata_path, 'rb') as f:
        metadata = pickle.load(f)
    
    print("=== t-SNE Analysis Summary ===")
    print(f"Pre-QC samples: {len(metadata['preqc']['anxiety']):,}")
    print(f"Post-QC samples: {len(metadata['postqc']['anxiety']):,}")
    print(f"QC removal: {len(metadata['preqc']['anxiety']) - len(metadata['postqc']['anxiety']):,} samples")
    print(f"t-SNE complexity: {complexity}")
    
    # Show anxiety group distribution
    import pandas as pd
    anxiety_counts = pd.Series(metadata['postqc']['anxiety']).value_counts()
    print(f"\nAnxiety group distribution: {anxiety_counts.to_dict()}")
    
else:
    print("Metadata not found. Run the full pipeline first.")

## Conclusion

This analysis demonstrates:

1. **Quality Control**: Effective removal of poor-quality samples with high surface topology defects
2. **Harmonization**: Reduction of scanner-related clustering while preserving anxiety-related patterns
3. **Demographics**: Consistent distribution of age and sex across the harmonized data

The t-SNE embeddings and plots are saved for further analysis and publication.